In [181]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
from src.utils import generate_test_data

In [189]:
y, X, Z, clusters, b, ptev, prev = generate_test_data([2, 1], m=1, sigma_b=2, sigma_e=1)

INFO     [utils.py:70] Drew 3 samples from 2 clusters.
INFO     [utils.py:71] PTEV = 91.13336148139933, PREV = 38.91720166784537.


In [190]:
y.head()

0    7.852193
1    3.301670
2    3.834297
dtype: float64

In [191]:
X.head()

,0,1,2
0,0.804728,1.473515,0.259276
1,0.271070,0.767424,-1.099978
2,-1.273002,-0.065982,0.235468


In [192]:
Z.head()

,0
0,1.0
1,1.0
2,1.0


In [193]:
clusters

0    0
1    0
2    1
dtype: int64

In [195]:
 # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Initialization ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n_clusters = clusters.nunique()
n_samples = len(y)
q = Z.shape[1]  # random effects dimension
# p = X.shape[1]  # fixed effects dimension

# Create a series where cluster_id is the index and n_i is the value
cluster_counts = clusters.value_counts()

# Do expensive slicing operations only once
Z_by_cluster = {}
y_by_cluster = {}
n_by_cluster = {}
I_by_cluster = {}
indices_by_cluster = {}

# TODO: Can these be replaced with groupbys? Groupbys are less understandable than brute force.
for cluster_id in cluster_counts.index:
    # Find the index for all the samples from this cluster in the large vector
    indices_i = (clusters == cluster_id)
    indices_by_cluster[cluster_id] = indices_i

    # Slice those samples from Z and y
    Z_by_cluster[cluster_id] = Z[indices_i]
    y_by_cluster[cluster_id] = y[indices_i]

    # Get the counts for each cluster and create the appropriately sized identity matrix for later computations
    n_by_cluster[cluster_id] = cluster_counts[cluster_id]
    I_by_cluster[cluster_id] = np.eye(cluster_counts[cluster_id])

# Intialize for EM algorithm
iteration = 0
b_hat = np.zeros((n_clusters, q))  # dimension is n_clusters X q
sigma2_hat = 1
D_hat = np.eye(q)

# vectors to hold history
b_hat_history = [b_hat]
sigma2_hat_history = [sigma2_hat]
D_hat_history = [D_hat]
gll_history = []

In [196]:
cluster_counts

0    2
1    1
dtype: int64

In [201]:
indices_by_cluster[0]

0     True
1     True
2    False
dtype: bool

In [207]:
y_by_cluster[1]

2    3.834297
dtype: float64

In [209]:
 # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ E-step ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# fill up y_star for all clusters
y_star = np.zeros(len(y))
for cluster_id in cluster_counts.index:
    # Get cached cluster slices
    y_i = y_by_cluster[cluster_id]
    Z_i = Z_by_cluster[cluster_id]
    b_hat_i = b_hat[cluster_id]
    indices_i = indices_by_cluster[cluster_id]

    # Compute y_star for this cluster and put back in right place
    y_star_i = y_i - Z_i.dot(b_hat_i)
    y_star[indices_i] = y_star_i

# check that still one dimensional
# TODO: Other checks we want to do?
assert(len(y_star.shape) == 1)

# Do the random forest regression with all the fixed effects features
rf = RandomForestRegressor(n_estimators=300, oob_score=True)
rf.fit(X, y_star)
f_hat = rf.oob_prediction_

In [211]:
y_star

array([ 7.85219332,  3.30167026,  3.83429745])

In [212]:
y

0    7.852193
1    3.301670
2    3.834297
dtype: float64

In [214]:
b_hat

array([[ 0.],
       [ 0.]])

In [215]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ M-step ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
sigma2_hat_sum = 0
D_hat_sum = 0

for cluster_id in cluster_counts.index:
    # Get cached cluster slices
    indices_i = indices_by_cluster[cluster_id]
    y_i = y_by_cluster[cluster_id]
    Z_i = Z_by_cluster[cluster_id]
    n_i = n_by_cluster[cluster_id]
    I_i = I_by_cluster[cluster_id]

    # index into f_hat
    f_hat_i = f_hat[indices_i]

    # Compute V_hat_i
    V_hat_i = Z_i.dot(D_hat).dot(Z_i.T) + sigma2_hat * I_i

    # Compute b_hat_i
    V_hat_inv_i = np.linalg.pinv(V_hat_i)
    b_hat_i = D_hat.dot(Z_i.T).dot(V_hat_inv_i).dot(y_i - f_hat_i)

    # Compute the total error for this cluster
    eps_hat_i = y_i - f_hat_i - Z_i.dot(b_hat_i)

    # Store b_hat for cluster
    b_hat[cluster_id] = b_hat_i

    # Update the sums for sigma2_hat and D_hat. We will update after the entire loop over clusters
    sigma2_hat_sum += eps_hat_i.T.dot(eps_hat_i) + sigma2_hat * (n_i - sigma2_hat * np.trace(V_hat_i))
    D_hat_sum += np.outer(b_hat_i, b_hat_i) + (D_hat - D_hat.dot(Z_i.T).dot(V_hat_inv_i).dot(Z_i).dot(D_hat))

# Normalize the sums to get sigma2_hat and D_hat
sigma2_hat = (1. / n_samples) * sigma2_hat_sum
D_hat = (1. / n_clusters) * D_hat_sum

In [216]:
b_hat

array([[ 0.41516769],
       [-0.51580755]])

In [217]:
D_hat

array([[ 0.63587749]])

In [218]:
sigma2_hat

8.4505129014160527

In [222]:
from src.merf import Merf

In [223]:
mrf = Merf(max_iterations=10)

In [224]:
mrf.fit(X, Z, clusters, y)

../src/merf.py:172: RuntimeWarning: invalid value encountered in log
../src/merf.py:172: RuntimeWarning: invalid value encountered in log
/Users/souravdey/.virtualenvs/merf/lib/python3.6/site-packages/numpy/linalg/linalg.py:1804: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
../src/merf.py:172: RuntimeWarning: invalid value encountered in log
../src/merf.py:139: RuntimeWarning: overflow encountered in double_scalars
  D_hat_history.append(D_hat)
../src/merf.py:167: RuntimeWarning: invalid value encountered in multiply
/Users/souravdey/.virtualenvs/merf/lib/python3.6/site-packages/numpy/linalg/linalg.py:1804: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
../src/merf.py:126: RuntimeWarning: invalid value encountered in multiply
  b_hat[cluster_id] = b_hat_i
/Users/souravdey/.virtualenvs/merf/lib/python3.6/site-packages/numpy/linalg/linalg.py:1804: RuntimeWarning: invalid value encountered in 

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').